# Imports

In [ ]:
# initial setup
import os
import asf_search as asf
from eof.download import download_eofs

# Settings

In [ ]:
# paths
data_dir = '/data'
download_folder = os.path.join(data_dir,'scenes')
credentials_path = "credentials/credentials_earthdata.txt"
precise_orb_download_folder = os.path.join(data_dir,'orbits','POEORB')
restituted_orb_download_folder = os.path.join(data_dir,'orbits','RESORB')
dem_download_folder = os.path.join(data_dir,'dem')

# Credentials

In [ ]:
# read credentials from file
with open(credentials_path, "r") as f:
   txt = str(f.read())
   uid = txt.split('\n')[1].split('login')[-1][1:]
   pswd = txt.split('\n')[2].split('password')[-1][1:]
   email = txt.split('\n')[3].split('email')[-1][1:]

# AOI

In [ ]:
wkt = "POINT (130.7085216 -16.0044816)" # South Australia
wkt = "POINT (110.526792 -66.282343)" # Casey station Antarctica
print(f'Searching over point: {wkt}')

# Search

### General time and area search

In [ ]:
# product type https://github.com/asfadmin/Discovery-asf_search/blob/master/asf_search/constants/PRODUCT_TYPE.py
# prod = 'GRD_HS' # IW
# prod = 'GRD_MD' # EW

# prod = 'GRD_HD'
# prod = 'GRD_MS'
# prod = 'GRD_FD'
prod = 'SLC'

# acquisition mode
mode = 'IW'
#mode = 'EW'

results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                     intersectsWith=wkt, 
                     maxResults=10, 
                     processingLevel=prod,
                     beamMode=mode,
                     start='2021-12-31T11:59:59Z',
                     end='2022-01-19T11:59:59Z')
print(len(results))

### Search for known products

In [ ]:
search_list = True
if search_list:

    prod = 'SLC'
    mode = 'IW'
    
    scene_list = ['S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63', 
   ]

    # search the scene list with the specified product type
    results = asf.granule_search(scene_list, asf.ASFSearchOptions(processingLevel=prod))

In [ ]:
for r in results:
    print(r.__dict__['umm']['GranuleUR'].split('-')[0])
    print(r.__dict__['umm']['AdditionalAttributes'][1]['Values'],
          r.__dict__['umm']['AdditionalAttributes'][17],
          r.__dict__['umm']['OrbitCalculatedSpatialDomains'][0]) #[1] #['FRAME_NUMBER']
    print(r.__dict__['umm']['SpatialExtent'])

# Download Scenes

In [ ]:
# download
session = asf.ASFSession()
session.auth_with_creds(uid,pswd)

In [ ]:
# download all results
scene_paths = []
scene_names = []
for scene in results:
    name = scene.__dict__['umm']['GranuleUR'].split('-')[0]
    scene_names.append(name)
    print(name)
    path = os.path.join(download_folder, name)
    scene.download(path=download_folder, session=session)
    scene_paths.append(path)

# Download Precise Orbit Files

In [ ]:
for scene in  scene_paths:
    # download precise orbits
    download_eofs(sentinel_file=scene, save_dir=precise_orb_download_folder, orbit_type='precise')
    # download restituted orbits
    download_eofs(sentinel_file=scene, save_dir=restituted_orb_download_folder, orbit_type='restituted')

In [ ]:
print('precise orbits')
for p in os.listdir(precise_orb_download_folder):
    print(os.path.join(precise_orb_download_folder,p))

print('restited orbits')
for p in os.listdir(restituted_orb_download_folder):
    print(os.path.join(restituted_orb_download_folder,p))

# DEMs

In [ ]:
# add your credentials to a .netrc file
# https://pypi.org/project/dem-stitcher/

In [ ]:
import rasterio
from dem_stitcher import stitch_dem
from shapely.geometry import Polygon

# valid DEM list here - # https://pypi.org/project/dem-stitcher/
dem_name = 'glo_30'  # Global Copernicus 30 meter resolution DEM

for r in results:
    scene_name = r.__dict__['umm']['GranuleUR'].split('-')[0]
    print(r.__dict__['umm']['GranuleUR'].split('-')[0])
    points = (r.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
              ['Geometry']['GPolygons'][0]['Boundary']['Points'])
    points = [(p['Longitude'],p['Latitude']) for p in points]
    poly = Polygon(points)
    #bounds = poly.bounds # as xmin, ymin, xmax, ymax in epsg:4326
    bounds = poly.buffer(1).bounds #buffered
    print(f'downloding DEM for scene bounds : {bounds}')
    # poly_3031 = transform_polygon(4326, 3031, poly, always_xy=True)
    # poly = antimeridian.fix_polygon(poly)
   
    # get the DEM and geometry information
    X, p = stitch_dem(bounds,
                    dem_name=dem_name,
                    dst_ellipsoidal_height=False,
                    dst_area_or_point='Point')

    # make folders and set filenames
    dem_dl_folder = os.path.join(dem_download_folder,dem_name)
    os.makedirs(dem_dl_folder, exist_ok=True)
    dem_filename = scene_name + '_dem.tif'
    save_path = os.path.join(dem_dl_folder,dem_filename)
    
    # save with rasterio
    print(f'saving dem to {save_path}')
    with rasterio.open(save_path, 'w', **p) as ds:
        ds.write(X, 1)
        ds.update_tags(AREA_OR_POINT='Point')